In [1]:
from lab3.experiment import base

## Working with experiment classes
This tutorial will give a quick survey of the experiment module. The major tools provided here are in `base`, which provides a set of objects to represent the most common types of experiments done in the lab. We'll point out the main functionality, and point out the major departures from the old repo's experiment class when necessary.

By the end of this notebook, you will know how to initialize `BehaviorExperiment` and `ImagingExperiment` objects, how to pair imaging data with trials in the experiment database, and how to access the raw behavior and imaging data through the experiment object properties and methods.

In [2]:
base?

Type:        module
String form: <module 'lab3.experiment.base' from '/home/james/code/lab3/lab3/experiment/base.py'>
File:        ~/code/lab3/lab3/experiment/base.py
Docstring:  
A collection of base classes for representing the basic experiment types
used in the lab: combinations of behavior, imaging, and LFP recordings.
Inherit from these to extend their functionality for more paradigms, e.g.
specific behavior tasks, that lend themselves to bespoke analysis methods.


### Using `BehaviorExperiment`

If you are familiar with the lab repo's `dbExperiment` class, this class works very similarly. It is initialized by passing an experiment ID from the sql database, and provides access to the behavior data and experiment attributes stored in the database. You can query the database to find the trial id of your experiment using `base.fetch_trials`.

In [3]:
base.BehaviorExperiment?

Init signature: base.BehaviorExperiment(trial_id)
Docstring:     
Base class for all experiments in the behavior database.

Parameters
----------
trial_id : int
    ID of experiment in the sql database
File:           ~/code/lab3/lab3/experiment/base.py
Type:           type
Subclasses:     ImagingExperiment, LFPExperiment


In [4]:
# e.g. find a trial_id by specifying some of the trial_info
trial_id = base.fetch_trials(project_name='james', mouse_name='jbp027', 
                             experimentType='delayed_associative_memory', 
                             session=0, condition=0, start_time='2018-10-05-17h53m45s')
print(trial_id)

[16716]


In [5]:
# initialize a BehaviorExperiment object
expt = base.BehaviorExperiment(trial_id)
print(expt)

<BehaviorExperiment: trial_id=[16716] mouse_id=572 experimentType=delayed_associative_memory>


### Accessing behavior data through `BehaviorExperiment` properties and methods 

Compared to the old repo experiment objects, the code for accessing and formatting behavior data is simplified. After running the tdml pickling script, behavior data is stored as a dictionary, where most variables of interest are stored as intervals (i.e. start/stop times). The `BehaviorExperiment` object exposes a `behavior_data` property, that returns this unformatted dictionary unchanged.

Often we would like to convert behavior data to indicator variables, that take values of True (when in the interval) or False (when not in the interval), and are sampled at regular intervals (e.g. to match the sampling of imaging data). To do this, use the `format_behavior_data()` method, which takes additional parameters to customize the sampling interval and other settings. This method and the previous property replace the overloaded functionality of `trial.behaviorData()` which was inherited by the old-style experiment class.

Another change from the old system is that velocity is calculated and included in the dictionary returned by `format_behavior_data()`. You can also return just the velocity via a separate method `velocity()`. Both `velocity()` and `format_behavior_data()` take a parameter `sigma` that controls the degree of smoothing in the velocity trace.

Examples of these methods and the documentation is shown below:

In [6]:
# This is the behavior data property
expt.behavior_data?

Type:        property
String form: <property object at 0x7f0c19e55a48>
Docstring:   Get unformatted behavior dictionary from pkl file.


In [7]:
# Take a look inside (note no parentheses)
beh_dict = expt.behavior_data

# note variables are stored as intervals (i.e. start/stop times)
print(beh_dict['odorA'])

[[33.86299896 36.86299896]]


In [8]:
# This is the behavior data formatting method
expt.format_behavior_data?

Signature:
expt.format_behavior_data(
    sampling_interval=0.1,
    discard_pre=0,
    discard_post=inf,
    sigma=0.1,
)
Docstring:
Format behavior dictionary. Interval variables are converted to
boolean vectors that denote activity at each sampling period.
Continuous variables are re-sampled at each discrete sampling period.

Parameters
----------
sampling_interval : float, optional
    Sampling rate for discretizing time. Variables expressed as
    intervals (i.e. vectors of start and stop times) are converted to
    binary vectors that are True for frames inside the intervals.
    Continuous variables are resampled at the corresponding time
    points.
discard_pre : {'first_lap', float}, optional
    If 'first_lap', truncate data before the first complete lap. If
    float, truncate data occuring before this time. Default behavior
    returns all data points
discard_post : float, optional
    Truncate data occuring after this time. Default behavior returns
    all data points
sigm

In [9]:
# Let's get the behavior dictionary with default parameters
beh_dict = expt.format_behavior_data()

In [10]:
print(beh_dict.keys())

dict_keys(['odorA_pin', 'lick', 'recordingDuration', 'trackLength', 'odorB', 'odorA', 'water', 'sync_pin', 'odorB_pin', 'treadmillPosition', 'reward_pin', 'lap', 'reward', 'velocity', 'lap_bin', 'sampling_interval', 'discard_pre', 'discard_post', 'json'])


In [11]:
# note the interval variables are binarized now 
print(beh_dict['odorA'])

[False False False ... False False False]


In [12]:
# and we can access velocity through this dictionary
print(beh_dict['velocity'])

[-25.02003921 -40.73965032 -45.25877341 ... -52.66123709 -51.7373391
 -44.91060497]


In [13]:
# or through the velocity method
print(expt.velocity())

[-25.02003921 -40.73965032 -45.25877341 ... -52.66123709 -51.7373391
 -44.91060497]


### Using `ImagingExperiment`

So far we've explored the BehaviorExperiment class, which provides methods for accessing information from the database and the underlying behavior data. This is class is already sufficient if the only data you are handling is behavior data, but more often we have concurrently recorded imaging data that we would like to analyze in parallel. So we'll augment this functionality to include methods specific to imaging data in the `ImagingExperiment` class.

Like `BehaviorExperiment`, `ImagingExperiment` is initialized by passing a `trial_id` from the database, but additionally we must pair this trial with a sima directory.

In [14]:
base.ImagingExperiment?

Init signature:
base.ImagingExperiment(
    trial_id,
    sima_path=None,
    force_pairing=False,
    store=False,
)
Docstring:     
Imaging experiment with behavior. If the imaging directory is not
already paired with the database trial id, or you would like to modify it,
this can be passed during initialization and the database will be updated
accordingly.

Parameters
----------
trial_id : int
    ID of experiment in the sql database
sima_path : str, optional
    Path to a sima folder. If passed, this imaging data will be paired in
    the database with this trial_id.
force_pairing : bool, optional
    If trial_id is already paired with a sima folder but tseries_path
    is passed, whether to overwrite pairing. Default is False
store : bool, optional
    Whether to store the newly paired sima_path in the database, if passed.
    Defaults to False.
File:           ~/code/lab3/lab3/experiment/base.py
Type:           type
Subclasses:     ImagingLFPExperiment


In [15]:
# if we try to initialize our current experiment, it will fail due to the lack of a sima path
expt = base.ImagingExperiment(trial_id)

AttributeError: Trial [16716] has no sima_path, or the path is not valid. Try passing a sima_path when initializing ImagingExperiment.

### Pairing imaging data

For trials that have not been paired with a sima directory yet, we can do the pairing during initialization of the `ImagingExperiment` object by passing the `sima_path` argument. We can also overwrite an existing pairing by passing `force_pairing=True`. Lastly, we can store the pairing in the database by passing `store=True`, so that in the future, we may instantiate the experiment object using just the trial_id without re-specifying the sima directory. This is similar to setting the `tSeries_path` attribute in the old experiment class, but here we require the user to unambiguously pair each experiment with a single sima directory, rather than a folder that may possibly contain multiple sima datasets.

For now, illustration, we'll just pair the data with the example dataset included in sima, and *not* store this information in the database.

In [18]:
# copy example sima dataset into working directory 

# from shutil import copy, copytree
# import sima.misc
# copytree(sima.misc.example_data(), 'example.sima')
# copy(sima.misc.example_tiff(), 'example.tif')
# copy(sima.misc.example_tiff(), 'example_Ch1.tif')
# copy(sima.misc.example_tiff(), 'example_Ch2.tif')
# copy(sima.misc.example_hdf5(), 'example.h5')

# this is the sima_path
sima_path = 'example.sima'

# instantiate an ImagingExperiment
expt = base.ImagingExperiment(trial_id, sima_path=sima_path)

# note this will print the changes that would be made to the database if store=True, 
# but will not make the changes unless we explicitly pass that argument during initialization

AssertionError: Not a valid sima path

### Properties of `ImagingExperiment`

There are many properties we can access now through the experiment object. Rather than go through them each, here is a list of the most commonly used ones:
- `frame_rate` - returns the frame rate of the imaging dataset
- `frame_period` - inverse of the frame rate
- `imaging_parameters` - returns the attribute table from the underlying h5 dataset
- `imaging_dataset` - returns the underlying sima `ImagingDataset` object
- `signals_path` - returns the path to the `signals.h5` file, which will contain extracted and processed traces from this dataset
- `suite2p_imaging_dataset` - returns a `Suite2pImagingDataset` object, which can be used to run Suite2p extraction and import routines on the dataset

Here are also the important methods for accessing the data. Note most of these methods will only work with new-style extracted traces:
- `signals_file()` - returns the `signals.h5` file containing traces
- `signals()` - returns a pandas DataFrame with the desired signals (e.g. raw, dfof, spikes for a given channel and label. This replaces the formed `imagingData()` and `spikes()` methods and provides a unified storage mechanism for time series data).
- `calculate_dfof()` - convenience method for running dF/F calculations and saving the signals
- `infer_spikes()` - convenience method for running spike inference and saving the signals

Lastly, `format_behavior_data()` and `velocity()` are augmented with an `image_sync` parameters (which is True by default), which automatically synchronzies the sampling of behavior data with the imaging frame period, and trims the behavior data to match the imaging data duration. 

**Note! Before using** `ImagingExperiment` **on older datasets, you should run the** `update_h5.py` **script located in** `lab3/scripts`, **which will add some additional metadata to the h5 file (namely the frame rate and imaging system). This simplifies some operations by storing this information permanently, rather than continuing to recalculate it as part of the experiment methods. This will eventually be integrated into an updated version of** `prairie2h5.py`

# TODO 
- Fill this in with examples of the properties and methods above (hard to do with the example sima dataset since it doesn't have the imaging parameters stored. Use one of my experiments as a permanent example?)
- Give an exampleo of ImagingOnlyExperiment and use cases
- Mention LFP classes?

KeyError: 'suite2p_imaging_dataset'